In [3]:
import numpy as np
import cv2
import glob
from matplotlib import pyplot as plt
import os
import pandas as pd
# from osgeo import gdal
# from osgeo import osr
import geopandas as gpd
from shapely.geometry import Point, Polygon, LineString

def imshow(image, show_axes = False, quiet = False):
    if len(image.shape) == 3:
      # Height, width, channels
      # Assume BGR, do a conversion since 
      image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    else:
      # Height, width - must be grayscale
      # convert to RGB, since matplotlib will plot in a weird colormap (instead of black = 0, white = 1)
      image = cv2.cvtColor(image, cv2.COLOR_GRAY2RGB)
    # Draw the image
    plt.imshow(image)
    if not show_axes:
        # We'll also disable drawing the axes and tick marks in the plot, since it's actually an image
        plt.axis('off')
    if not quiet:
        # Make sure it outputs
        plt.show()

# Goal:
This note book use detected (tracked) person with individual attributes to aggregate data

In [ ]:
def get_proj_matrix(ref):
    '''

    pts_src and pts_dst are numpy arrays of points

    in source and destination images. We need at least

    corresponding points.

    '''
    try:
        pts_src = np.array([(x,y) for x,y in zip(ref['sourceX'], -1*ref['sourceY'])])
    except:
        pts_src = np.array([(x,y) for x,y in zip(ref['pixelX'], -1*ref['pixelY'])])

    pts_dst  = np.array([(x,y) for x,y in zip(ref['mapX'], ref['mapY'])])

    h, status = cv2.findHomography(pts_src, pts_dst)

    '''
    The calculated homography can be used to warp

    the source image to destination. Size is the

    size (width,height) of im_dst
    '''
    return h

def projectPlan(df, h, x, y):
    pts = df[[x, y]].values
    ## (n, 1, 2)
    pts1 = pts.reshape(-1,1,2).astype(np.float32)
    dst1 = cv2.perspectiveTransform(pts1, h)
    return dst1


def pixel2coord(col, row, ds):
    # 3. transform to 2326 geolocation
    c, a, b, f, d, e = ds.GetGeoTransform()
    """Returns global coordinates to pixel center using base-0 raster index"""
    xp = a * col + b * row + a * 0.5 + b * 0.5 + c
    yp = d * col + e * row + d * 0.5 + e * 0.5 + f
    return(xp, yp)


# bbox0, bbox1, bbox2, bbox3 : x1, y1, w, h
# Replace this part for other data
# Trace

# check gender distribution within one track_id
# set attribute list
# def get_attr(trace):
#     attribute_ls = ['gender', 'age', 'side', 'glasses', 'hat', 'hold_objects_in_front',
#         'bag', 'upper', 'lower', 'boots']
#     # for each track_id only keep one major attribute
#     for attr in attribute_ls:
#         trace[attr] = trace.groupby("track_id")[attr].transform(lambda x: x.mode()[0])
#     trace[trace["track_id"] == 1].groupby(["gender"]).size()

#     attr_df = trace.drop_duplicates("track_id")[attribute_ls+["track_id"]]
#     return attr_df

def getclean(trace, h, epsg, videoname):

    trace['loc_x'] = (trace['bbox0'] + trace['bbox0'] + trace['bbox2'])/2
    trace['loc_y'] = (trace['bbox1'] + trace['bbox3'])
    
    trs2 = projectPlan(trace, h, 'loc_x', 'loc_y')
    trace[f'x_{epsg}'] = trs2[:,:,0]
    trace[f'y_{epsg}'] = trs2[:,:,1] 
    
    trace['video_id'] = videoname

        # smoothe the x, y for every 30 frames
    attribute_ls = ['gender', 
                    'age', 
                    'side', 
                    'glasses', 
                    'hat', 
                    'hold_objects_in_front',
                    'bag', 
                    'upper', 
                    'lower', 
                    'boots']
    
    cols = ['video_id',
        'frame_id',
                  'track_id','loc_x', 'loc_y',
                     f'x_{epsg}', f'y_{epsg}', # reference geo in HK
                     'category_id',
                     "score"
                  ]
    cols_keep = [x for x in trace.columns if x in cols+attribute_ls]
    return trace[cols_keep]
            

def getgdf(traceDF, epsg, tail = True, length = 3):
    """length: refers to the second of lagging tail we want to see"""
    # smoothe the x, y for every 30 frames
    traceDF['moving_x'] = traceDF.groupby('track_id')[f'x_{epsg}'].transform(lambda x: x.rolling(5, 1).mean())
    traceDF['moving_y'] = traceDF.groupby('track_id')[f'y_{epsg}'].transform(lambda x: x.rolling(5, 1).mean())

    traceGDF = gpd.GeoDataFrame(traceDF, geometry = [Point(x,y) for x,y in zip(traceDF[f'x_{epsg}'],
                                                                               traceDF[f'y_{epsg}'])])
    traceGDF.crs = f"EPSG:{epsg}"
    traceGDF = traceGDF.to_crs('EPSG:4326')
    traceGDF['lat'] = traceGDF['geometry'].y
    traceGDF['lon'] = traceGDF['geometry'].x
    
    traceGDF['lat_moving'] = traceGDF.groupby('track_id')['lat'].transform(lambda x: x.rolling(5, 1).mean())
    traceGDF['lon_moving'] = traceGDF.groupby('track_id')['lon'].transform(lambda x: x.rolling(5, 1).mean())
    return traceGDF

# drop the outlier automatically
def find_outliers_IQR(df, field, low = 0.25, high = 0.75):

   q1=df[field].quantile(low)

   q3=df[field].quantile(high)

   IQR=q3-q1

   outliers = df[((df[field]<(q1-1.5*IQR)) | (df[field]>(q3+1.5*IQR)))]

   keep = df[((df[field]>=(q1-1.5*IQR)) & (df[field]<=(q3+1.5*IQR)))].reset_index(drop = True)

   return outliers, keep


In [ ]:
def get_proj_video(videoname, ref):
    """For new videos, we extract the detection results from tracks"""
    result_folder = "../../_data/03_tracking_result/_current_attr_result"
    predpath = os.path.join(result_folder, f'{videoname}.csv')
    trace = pd.read_csv(predpath)
    trace['ratio'] = trace['w']/trace['h']
    _, trace = find_outliers_IQR(trace, 'ratio', 0.15, 0.85)
    trace.rename(columns = {"x":"bbox0", "y":"bbox1", "w":"bbox2", "h":"bbox3"}, inplace = True)
    h = get_proj_matrix(ref)
# # Set up projection for New York State Plane
# set up to match the projection of the reference data
    # epsg = 3857  #2263
    epsg = videopath_sel[videopath_sel['video_id']==videoname]['ref_epsg'].values[0]
    traceDF = getclean(trace, h, epsg, videoname)
    traceGDF = getgdf(traceDF, epsg)
    
    attr_df = get_attr(traceDF)

    # drop outliers
    _, traceGDF_keep = find_outliers_IQR(traceGDF, "moving_x")
    
    traceGDF = traceGDF_keep.drop(["gender","age"], axis = 1).merge(attr_df[["track_id", "gender", "age"]], 
                                                                    on = "track_id", how = "left")
    
    return traceGDF

def getbasics(file_path):
    video = cv2.VideoCapture(file_path)
    fps = video.get(cv2.CAP_PROP_FPS)
    print('frames per second =',fps)
    size = (int(video.get(cv2.CAP_PROP_FRAME_WIDTH)), int(video.get(cv2.CAP_PROP_FRAME_HEIGHT)))
    print('frames size =',size)
    # video.release()
    return video, fps, size

# read in the points
def get_ref(ref_path):
    # with open(ref_folder + ref_video + f"_3446_modified.tif.points", "r") as f:
    with open(os.path.join(ref_folder, ref_path), "r") as f:
        lines = f.readlines()
        lines = [line.strip().split(",") for line in lines]
    # convert to dataframe
    ref = pd.DataFrame(lines[1:], columns = lines[0])
    # convert to float
    ref = ref.astype(float)
    return ref


def get_all_info(videoname, useimage = True):
    # frame_folder = "../_data/02_siteplan/sample_frames"
    # tiff_folder = "../_data/02_siteplan/geo_tiff"
    # ref_folder = "../_data/02_siteplan/gcp_pt/"

    # ref = pd.read_csv(os.path.join(ref_folder, f'{videoname}_reference.txt'), sep = ",")
    ref_path = videopath_sel[videopath_sel['video_id']==videoname]['ref_path'].values[0]
    ref = get_ref(ref_path)
    # targetimg  = cv2.cvtColor(cv2.imread(os.path.join(tiff_folder, f'{videoname}_modified.tif')), cv2.COLOR_BGR2RGB)
    
    #===================== PLOT the reference points =====================
    # originimg  = cv2.cvtColor(cv2.imread(os.path.join(frame_folder, f'{videoname}_{frame_start}.jpg')), cv2.COLOR_BGR2RGB)

    # fig, ax = plt.subplots(figsize = (10,10))
    # plt.imshow(originimg)
    # try:
    #     plt.scatter(ref['pixelX'], -1*ref['pixelY'], color = 'red')
    # except:
    #     plt.scatter(ref['sourceX'], -1*ref['sourceY'], color = 'red')
    # if useimage:
    #     traceGDF_keep = get_proj_img(videoname, ref)
    # else:
    traceGDF_keep = get_proj_video(videoname, ref)
    if "category_id" in traceGDF_keep.columns:
        traceGDF_people = traceGDF_keep[(traceGDF_keep["category_id"] == 0)&(traceGDF_keep["score"]>0.1)].reset_index(drop = True)
        return traceGDF_people
    else:
        return traceGDF_keep
    
def get_frame_num(time_str, fps = 29.97002997002997):
    try:
        time = time_str.split(" ")[0][3:].split(":")
        minute = int(time[0])
        second = int(time[1])
        frame = minute*60*fps + second*fps
        return int(frame)
    except:
        return np.nan

In [ ]:
from tqdm import tqdm
outputfolder = "../../_data/05_tracking_result_projected/step0_attr_prj"
frame_folder = "../../_data/02_siteplan/sample_frames/current_sample"
tiff_folder = "../../_data/02_siteplan/geo_tiff"
ref_folder = "../../_data/02_siteplan/gcp_pt/"
frames = os.listdir(frame_folder)
refs = os.listdir(ref_folder)


In [ ]:
videopath = pd.read_csv("../../_data/00_raw/_video_meta/video_path_0509.csv")

# videopath_sel = videopath[videopath['scene'].isin([2,3])]
videopath_sel = videopath[~videopath['ref_path'].isna()].reset_index(drop = True)
videopath_sel['first_effective_time'].unique()
videopath_sel['first_effective_time'] = videopath_sel['first_effective_time'].fillna("12:00:00 AM")
videopath_sel['frame_start'] = videopath_sel['first_effective_time'].apply(lambda x: get_frame_num(x))

# videopath_sel['last_effective_time'] = videopath_sel['last_effective_time'].fillna("12:00:00 AM")
videopath_sel['frame_end'] = videopath_sel['last_effective_time'].apply(lambda x: get_frame_num(x))
videopath_sel['frame_end'] = videopath_sel['frame_end'].fillna(videopath_sel['length'])
videopath_sel['ref_epsg'] = videopath_sel['ref_epsg'].astype(int)

videopath_sel.shape

In [ ]:
videopath[(videopath['finished']==True)&(videopath['ref_path'].isna())].groupby("video_location").size()

In [ ]:
# list all projected csv
projected = glob.glob(outputfolder+"/*/*_projected.csv")
projected_names = [x.split("\\")[-1].split("_")[0] for x in projected]
# now_processing = videopath_sel[~videopath_sel['video_id'].isin(projected_names)].reset_index(drop = True)
now_processing = videopath_sel.copy()
now_processing = now_processing[now_processing['video_location']!='Met Steps videos (NEW)'].reset_index(drop = True)
now_processingls = now_processing['video_id'].unique().tolist()
len(now_processingls)

In [ ]:

for videoname in tqdm(now_processingls):
    print(videoname)
    try:
        traceGDF_people = get_all_info(videoname,  
                                    useimage = False)
        file_path = videopath[videopath['video_id'] == videoname]['videopath'].values[0]
        # video, fps, size = getbasics(file_path)
        fps = videopath[videopath['video_id'] == videoname]['fps'].values[0]
        
        # only keep the frame_id after the first desired frame
        first_frame_sel = videopath_sel[videopath_sel['video_id']== videoname]['frame_start'].values[0]
        last_frame_sel = videopath_sel[videopath_sel['video_id']== videoname]['frame_end'].values[0]
        traceGDF_people = traceGDF_people[(traceGDF_people['frame_id']<=last_frame_sel)&(traceGDF_people['frame_id']>=first_frame_sel)].reset_index(drop = True)
        
        traceGDF_people["second"] = traceGDF_people["frame_id"]//fps
        # sample = traceGDF_people[traceGDF_people["second"]<20] # pick 20 seconds sample
        loc_name = videopath[videopath['video_id'] == videoname]['video_location'].values[0]
        destfolder = os.path.join(outputfolder, loc_name)
        if not os.path.exists(destfolder):
            os.makedirs(destfolder)
        traceGDF_people.drop("geometry", axis = 1).to_csv(os.path.join(destfolder, 
                                                                       f"{videoname}_projected.csv"), index = False)
    except:
        print(f"error in {videoname}")

In [ ]:
# videoname = now_processingls[0]
# loc_name = videopath[videopath['video_id'] == videoname]['video_location'].values[0]
# destfolder = os.path.join(outputfolder, loc_name)

# traceGDF_people = pd.read_csv(os.path.join(destfolder, f"{videoname}_projected.csv"))
# traceGDF_people.head()

## test draw image to see the detection results

In [ ]:

# historical videofolder:
# file_path = f"../_data/00_raw/_mp4/videos_old_highres/{videoname}.mp4"
# current videofolder:
# file_path = f"../_data/00_raw/videos_current_highres/bryant_park/{videoname}.avi"
# videoname = "20100519-083343b11"
videopath = pd.read_csv("../../_data/00_raw/_video_meta/video_path.csv")
videopath['video_id'] = videopath['video_name'].apply(lambda x: x.split(".")[0])
file_path = videopath[videopath['video_id'] == videoname]['videopath'].values[0]

loc = videopath[videopath['video_id'] == videoname]['video_location'].values[0]
video, fps, size = getbasics(file_path)
frame_id = 350
video.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
ret, frame = video.read()
# plot the frame
frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
fig = plt.subplots(figsize = (10,10))
plt.imshow(frame)

# load traceGDF_people for this video
destfolder = os.path.join(outputfolder, loc)
filepath = os.path.join(destfolder, f"{videoname}_projected.csv")
traceGDF_people = pd.read_csv(filepath)
temp = traceGDF_people[traceGDF_people["frame_id"] == frame_id]
plt.scatter(
    temp["loc_x"], 
    temp["loc_y"], color = 'red')

# export the x,y coordinates to csv of this one frame
# temp[["geometry","track_id"]].to_file(f"../_data/05_demo/2023-04-30/{videoname}_frame_{frame_id}.geojson", 
# driver = "GeoJSON")

In [ ]:
temp = gpd.GeoDataFrame(temp, geometry = gpd.points_from_xy(temp["lon"], temp["lat"]))
temp.crs = "EPSG:4326"
temp[["geometry","track_id"]].to_file(f"../_data/05_demo/2023-04-30/{videoname}_frame_{frame_id}.geojson", 
driver = "GeoJSON")

# 2. Visualize the track examples

In [ ]:
videopath = pd.read_csv("../_data/00_raw/_video_meta/video_path.csv")
outputfolder = "../_data/05_tracking_result_projected/step0_attr_prj"
now_processing = videopath_sel[videopath_sel["finished"]==True].reset_index(drop = True)
# select one video for trace visualization
videoname = now_processing['video_id'][1]
destfolder = os.path.join(outputfolder, "bryant_park")
trace = pd.read_csv(os.path.join(destfolder, f"{videoname}_projected.csv"))
traceGDF = gpd.GeoDataFrame(trace, geometry=gpd.points_from_xy(trace.lon, trace.lat))

In [ ]:
epsg = 3857

# plot a trace sample
temp = traceGDF[traceGDF["frame_id"]<1000]
# construct lines
tracesum = temp.groupby("track_id").size().reset_index().rename(columns = {0:"count"})
trackidls = tracesum[tracesum["count"]>10]["track_id"].unique()
geo_df2 = temp[temp["track_id"].isin(trackidls)].sort_values("frame_id").reset_index(drop = True)\
.groupby(['track_id',"gender", "age"])['geometry'].apply(lambda x: LineString(x.tolist())).reset_index()

geo_df2.crs = "EPSG:4326"
geo_df2 = geo_df2.to_crs(f"EPSG:{epsg}")

In [ ]:
# assert and print pass
assert geo_df2.crs == f"EPSG:{epsg}", "crs is not correct"

geo_df2.plot(column = "gender", figsize = (10,10))

# Export at h3 level for occupation rate estimate

In [ ]:
fps = 29
traceGDF["second_from_start"] = traceGDF["frame_id"]/fps

traceGDF["minute"] = traceGDF["second_from_start"]//60
traceGDF["hour"] = traceGDF["second_from_start"]//3600
traceGDF["second"] = traceGDF["second_from_start"]- traceGDF["hour"]*3600 - traceGDF["minute"]*60
traceGDF["timestamp"] = "2008-10-08"+" " + traceGDF["hour"].astype(str) + ":"+traceGDF["minute"].astype(str).str.zfill(2)\
    +":"+traceGDF["second"].astype(str)
traceGDF["timestamp"] = pd.to_datetime(traceGDF["timestamp"])


# Convert all dectection to hexagon 15 for aggregation

In [ ]:
# traceGDF.to_file(os.path.join(clipfolder, f"{videoname[:-4]}_prediction_with_attr.geojson"), driver = "GeoJSON")

In [ ]:
traceGDF = pd.read_csv(r"D:\Dropbox (MIT)\whyte_CV\_data\05_tracking_result_projected\step0_attr_prj\20081008-141944b03_projected_with_attr.csv")
traceGDF.head()

In [ ]:
from h3 import h3
res = 15
traceGDF[f"h3_{res}"] = traceGDF.apply(lambda row: h3.geo_to_h3(row["lat"], row["lon"], res), axis = 1)


In [ ]:
# count people per h3 id per minute per gender

countpeople_gender = traceGDF.groupby([f"h3_{res}","gender"])["track_id"].nunique().reset_index()\
    .pivot(columns = "gender", index = ["h3_15"], values = "track_id").reset_index().fillna(0)
countpeople_gender
# countpeople_gender.to_csv(os.path.join(outfolder, f"{videoname}_prediction_aggregation.csv"), index = False)

In [ ]:
countpeople_age = traceGDF.groupby([f"h3_{res}","age"])["track_id"].nunique().reset_index()\
    .pivot(columns = "age", index = ["h3_15"], values = "track_id").reset_index().fillna(0)
countpeople_age

In [ ]:
summary = countpeople_gender.merge(countpeople_age, on = ["h3_15"], how = "outer")
summary["total"] = summary["Female"]+summary["Male"]
outfolder = "../_data/05_tracking_result_projected"
summary.to_csv(os.path.join(outfolder, f"{videoname}_prediction_aggregation_overall.csv"), index = False)

In [ ]:
summary = traceGDF_keep.groupby("track_id").size().reset_index()
summary[summary[0]>fps].shape


In [ ]:
countpeople = traceGDF_keep.groupby([f"h3_{res}"])["track_id"].nunique().reset_index()
countpeople.to_csv(os.path.join(clipfolder, f"{videoname}_prediction_aggregation_all.csv"), index = False)